# As supervised problem

In [38]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/rakeshbhatia/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/en_core_web_sm -->
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [39]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [40]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [41]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [42]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [43]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,intervention,hint,speak,St,dealing,friend,mode,case,introduction,beast,...,ay,office,injustice,wrinkle,publicly,consult,rely,rejoin,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [44]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9774436090225563

Test set score: 0.8146067415730337


/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [45]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1613) (266,)
Training set score: 0.9699248120300752

Test set score: 0.8764044943820225


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [46]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9661654135338346

Test set score: 0.8146067415730337


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [47]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma[:int(len(emma)/60)])
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [48]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [49]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [50]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 50
Processing row 100
Processing row 150
done


In [51]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.7235772357723578


col_0,Austen,Carroll
row_0,,
Austen,158,12
Carroll,56,20


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0
For this exercise, we will create new features for the following characteristics:
* Total adjectives
* Total adverbs
* Total determiners
* Total nouns
* Total numerals
* Total pronouns
* Total punctuation
* Total symbols
* Total verbs
* Total phrases

In [52]:
# Create copy of original DataFrame
wc = word_counts

# Create total words feature
wc['total_words'] = wc['text_sentence'].map(lambda x: len(x))

# Create new feature lists
adj_count = []
adv_count = []
det_count = []
noun_count = []
num_count = []
pron_count = []
punct_count = []
sym_count = []
verb_count = []
phrase_count = []

sentences = wc.text_sentence

for sentence in sentences:
    adjs, advs, dets, nouns, nums, prons, puncts, syms, verbs = 0, 0, 0, 0, 0, 0, 0, 0, 0
    
    # Check POS of each word in sentence
    for token in sentence:
        if token.pos_ == 'ADJ':
            adjs += 1
        elif token.pos_ == 'ADV':
            advs += 1
        elif token.pos_ == 'DET':
            dets += 1
        elif token.pos_ == 'NOUN':
            nouns += 1
        elif token.pos_ == 'NUM':
            nums += 1
        elif token.pos_ == 'PRON':
            prons += 1
        elif token.pos_ == 'PUNCT':
            puncts += 1
        elif token.pos_ == 'SYM':
            syms += 1
        elif token.pos_ == 'VERB':
            verbs += 1

        # Get chunks (phrases)
        chunks = [chunk.text for chunk in wc.text_sentence]

        # Add counts to appropriate list
        adj_count.append(adjs)
        adv_count.append(advs)
        det_count.append(dets)
        noun_count.append(nouns)
        num_count.append(nums)
        pron_count.append(prons)
        punct_count.append(puncts)
        sym_count.append(syms)
        verb_count.append(verbs)
        phrase_count.append(len(chunks))

# Create new features using the lists
wc['adj_count'] = pd.Series(adj_count)
wc['adv_count'] = pd.Series(adv_count)
wc['det_count'] = pd.Series(det_count)
wc['noun_count'] = pd.Series(noun_count)
wc['num_count'] = pd.Series(num_count)
wc['pron_count'] = pd.Series(pron_count)
wc['punct_count'] = pd.Series(punct_count)
wc['sym_count'] = pd.Series(sym_count)
wc['verb_count'] = pd.Series(verb_count)
wc['phrase_count'] = pd.Series(phrase_count)

wc.head()

,intervention,hint,speak,St,dealing,friend,mode,case,introduction,beast,...,adj_count,adv_count,det_count,noun_count,num_count,pron_count,punct_count,sym_count,verb_count,phrase_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,444
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,444
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,444
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,444
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,444


In [53]:
wc.describe()

,total_words,adj_count,adv_count,det_count,noun_count,num_count,pron_count,punct_count,sym_count,verb_count,phrase_count
count,444.000000,444.000000,444.000000,444.000000,444.000000,444.0,444.000000,444.000000,444.0,444.000000,444.0
mean,29.225225,0.851351,2.596847,2.495495,2.912162,0.0,2.018018,2.527027,0.0,4.425676,444.0
std,26.815963,1.342305,2.074697,2.192225,2.921692,0.0,1.816563,2.139954,0.0,3.483125,0.0
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,444.0
25%,10.750000,0.000000,1.000000,1.000000,1.000000,0.0,0.000000,1.000000,0.0,2.000000,444.0
50%,21.500000,1.000000,2.000000,2.000000,2.000000,0.0,2.000000,2.000000,0.0,4.000000,444.0
75%,39.000000,1.000000,3.000000,4.000000,5.000000,0.0,3.000000,3.000000,0.0,6.000000,444.0
max,187.000000,6.000000,10.000000,8.000000,12.000000,0.0,6.000000,10.000000,0.0,13.000000,444.0


In [54]:
from sklearn.svm import SVC

y = wc['text_source']
X = np.array(wc.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

svm = SVC(kernel = 'linear')                
train = svm.fit(X_train, y_train)

print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.9962406015037594

Test set score: 0.9382022471910112


Using Support Vector Machines (SVM) on our new set of features yielded solid accuracy scores of 99.62% for the training set and 93.82% for the test set. Thus, SVM seems to be a good choice of model for this dataset, as we were able to exceed 93% accuracy with the test set.

# Challenge 1
Let's print all the available novels.

In [55]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [56]:
# Clean the Melville data.
moby = gutenberg.raw('melville-moby_dick.txt')
moby = re.sub('ETYMOLOGY.', '', moby)
print(moby[:1000])

[Moby Dick by Herman Melville 1851]




(Supplied by a Late Consumptive Usher to a Grammar School)

The pale Usher--threadbare in coat, heart, body, and brain; I see him
now.  He was ever dusting his old lexicons and grammars, with a queer
handkerchief, mockingly embellished with all the gay flags of all the
known nations of the world.  He loved to dust his old grammars; it
somehow mildly reminded him of his mortality.

"While you take in hand to school others, and to teach them by what
name a whale-fish is to be called in our tongue leaving out, through
ignorance, the letter H, which almost alone maketh the signification
of the word, you deliver that which is not true." --HACKLUYT

"WHALE. ... Sw. and Dan. HVAL.  This animal is named from roundness
or rolling; for in Dan. HVALT is arched or vaulted." --WEBSTER'S
DICTIONARY

"WHALE. ... It is more immediately from the Dut. and Ger. WALLEN;
A.S. WALW-IAN, to roll, to wallow." --RICHARDSON'S DICTIONARY

KETOS,   


For this exercise, we will see whether our new model is good at identifying Carroll's *Alice in Wonderland* vs Melville's *Moby Dick*.

In [57]:
moby = gutenberg.raw('melville-moby_dick.txt')
moby = re.sub('ETYMOLOGY.', '', moby)
moby = re.sub('EXTRACTS', '', moby)
moby = re.sub('CHAPTER', '', moby)
moby = re.sub('Epilogue', '', moby)
moby = text_cleaner(moby[:int(len(moby)/50)])
print(moby[:1000])

(Supplied by a Late Consumptive Usher to a Grammar School) The pale Usher threadbare in coat, heart, body, and brain; I see him now. He was ever dusting his old lexicons and grammars, with a queer handkerchief, mockingly embellished with all the gay flags of all the known nations of the world. He loved to dust his old grammars; it somehow mildly reminded him of his mortality. "While you take in hand to school others, and to teach them by what name a whale-fish is to be called in our tongue leaving out, through ignorance, the letter H, which almost alone maketh the signification of the word, you deliver that which is not true." HACKLUYT "WHALE. ... Sw. and Dan. HVAL. This animal is named from roundness or rolling; for in Dan. HVALT is arched or vaulted." WEBSTER'S DICTIONARY "WHALE. ... It is more immediately from the Dut. and Ger. WALLEN; A.S. WALW-IAN, to roll, to wallow." RICHARDSON'S DICTIONARY KETOS, GREEK. CETUS, LATIN. WHOEL, ANGLO-SAXON. HVALT, DANISH. WAL, DUTCH. HWAL, SWEDISH.

In [58]:
moby_doc = nlp(moby)

In [59]:
# Extract sentences
moby_sents = [[sent, "Melville"] for sent in moby_doc.sents]
moby_sents = moby_sents[0:len(alice_sents)]

In [60]:
# Save sentences to DataFrame
moby_sentences = pd.DataFrame(moby_sents)
moby_bow = bow_features(moby_sentences, common_words)

Processing row 0
Processing row 50
Processing row 100


In [61]:
# Save sentences to DataFrame
alice_sentences = pd.DataFrame(alice_sents)
alice_bow = bow_features(alice_sentences, common_words)

Processing row 0
Processing row 50
Processing row 100


In [62]:
moby_test = moby_bow.drop(['text_sentence','text_source'], axis=1)
alice_test = alice_bow.drop(['text_sentence','text_source'], axis=1)
X = np.concatenate((moby_test, alice_test), axis=0)
y = np.concatenate((moby_bow['text_source'], alice_bow['text_source']), axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

### Random Forest

In [63]:
rfc = ensemble.RandomForestClassifier()
train = rfc.fit(X_train, y_train)
print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.961038961038961

Test set score: 0.7788461538461539


/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


We were able to achieve an accuracy of 94.8% with our training data, but our test set only yielded an accuracy of 77.9%, indicating overfitting.

In [67]:
from sklearn.model_selection import cross_val_score
rfc_scores = cross_val_score(rfc, X_test, y_test, cv=10)
print('Cross-Validation Scores:', rfc_scores)
print('Mean Cross-Validation Score: {:.2%} +/- {:.2%}'.format(rfc_scores.mean(), rfc_scores.std()))

Cross-Validation Scores: [1.         0.90909091 0.72727273 0.8        0.7        0.8
 0.7        0.6        0.7        0.7       ]
Mean Cross-Validation Score: 76.36% +/- 11.13%


In [64]:
y_pred = rfc.predict(X_test)
pd.crosstab(y_test, y_pred)

col_0,Carroll,Melville
row_0,,
Carroll,29,22
Melville,1,52


In [65]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Carroll       0.97      0.57      0.72        51
    Melville       0.70      0.98      0.82        53

    accuracy                           0.78       104
   macro avg       0.83      0.77      0.77       104
weighted avg       0.83      0.78      0.77       104



Our cross-validation accuracy scores were very inconsistent, with an average of 76.36% and a huge standard deviation of 11.13%.

In [66]:
#X_test = np.concatenate((moby_test, alice_test), axis=0)
#y_test = np.concatenate((moby_bow['text_source'], alice_bow['text_source']), axis=0)

### Logistic Regression

In [68]:
lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.948051948051948

Test set score: 0.8076923076923077


/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Using Logistic Regression, both our training and test set accuracy scores improved noticeably to 94.81% and 80.77%, respectively.

In [69]:
from sklearn.model_selection import cross_val_score
lr_scores = cross_val_score(lr, X_test, y_test, cv=10)
print('Cross-Validation Scores:', lr_scores)
print('Mean Cross-Validation Score: {:.2%} +/- {:.2%}'.format(lr_scores.mean(), lr_scores.std()))

Cross-Validation Scores: [0.91666667 0.81818182 0.81818182 0.8        0.7        0.7
 0.7        0.6        0.8        0.8       ]
Mean Cross-Validation Score: 76.53% +/- 8.50%


/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/rakeshbhatia/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default so

In [70]:
y_pred = lr.predict(X_test)
pd.crosstab(y_test, y_pred)

col_0,Carroll,Melville
row_0,,
Carroll,32,19
Melville,1,52


In [71]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Carroll       0.97      0.63      0.76        51
    Melville       0.73      0.98      0.84        53

    accuracy                           0.81       104
   macro avg       0.85      0.80      0.80       104
weighted avg       0.85      0.81      0.80       104



Cross-validation also performed better than Random Forest, with a slight improvement in average cross-validation score as well as lower standard deviation.

### Gradient Boosting

In [72]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.961038961038961

Test set score: 0.7596153846153846


Using Gradient Boosting, our training set accuracy improved to 96.1%, but our test set accuracy dropped to 75.96%, indicating overfitting.

In [73]:
from sklearn.model_selection import cross_val_score
clf_scores = cross_val_score(clf, X_test, y_test, cv=10)
print('Cross-Validation Scores:', clf_scores)
print('Mean Cross-Validation Score: {:.2%} +/- {:.2%}'.format(clf_scores.mean(), clf_scores.std()))

Cross-Validation Scores: [1.         0.90909091 0.63636364 0.8        0.6        0.7
 0.7        0.7        0.6        0.7       ]
Mean Cross-Validation Score: 73.45% +/- 12.48%


In [74]:
y_pred = clf.predict(X_test)
pd.crosstab(y_test, y_pred)

col_0,Carroll,Melville
row_0,,
Carroll,28,23
Melville,2,51


In [75]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     Carroll       0.93      0.55      0.69        51
    Melville       0.69      0.96      0.80        53

    accuracy                           0.76       104
   macro avg       0.81      0.76      0.75       104
weighted avg       0.81      0.76      0.75       104



Cross-validation here also didn't perform as well here, with an average cross-validation accuracy of only 73.45% and a high standard deviation of 12.48%. Thus, Logistic Regression proves to be the best model of the three, possessing the highest average cross-validation score and lowest standard deviation.